<a href="https://colab.research.google.com/github/jaya-shankar/education-impact/blob/master/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jaya-shankar/education-impact.git


Cloning into 'education-impact'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 40 (delta 11), reused 33 (delta 8), pack-reused 0
Unpacking objects: 100% (40/40), done.


In [ ]:
root = "education-impact/" 
datasets_path = {
                    "child_mortality"               :  root+ "datasets/child_mortality_0_5_year_olds_dying_per_1000_born.csv",
                    "children_per_woman"            :  root+"datasets/children_per_woman_total_fertility.csv",
                    "co2_emissions"                 :  root+"datasets/co2_emissions_tonnes_per_person.csv",
                    "population"                    :  root+ "datasets/converted_pop.csv",
                    "expenditure_per_student"       :  root+ "datasets/expenditure_per_student_secondary_percent_of_gdp_per_person.csv",
                    "food_supply"                   :  root+ "datasets/food_supply_kilocalories_per_person_and_day.csv",
                    "gdp_per_captia"                :  root+ "datasets/gdp_per_capita_yearly_growth.csv",
                    "gini_index"                    :  root+ "datasets/gini.csv",
                    "infant_mortality"              :  root+ "datasets/Infant_Mortality_Rate.csv",
                    "life_expectancy"               :  root+ "datasets/life_expectancy_years.csv",
                    "malnutrition"                  :  root+ "datasets/malnutrition_weight_for_age_percent_of_children_under_5.csv",
                    "poverty_index"                 :  root+ "datasets/mincpcap_cppp.csv",
                    "maternal_mortality"            :  root+ "datasets/mmr_who.csv",
                    "people_in_poverty"             :  root+ "datasets/number_of_people_in_poverty.csv",
                    "primary_completion"            :  root+ "datasets/primary_school_completion_percent_of_girls.csv",
                    "ratio_b/g_in_primary"          :  root+ "datasets/ratio_of_girls_to_boys_in_primary_and_secondary_education_perc.csv",
                    "sex_ratio"                     :  root+ "datasets/sex-ratio.csv"
                }

In [ ]:
datasets_path

{'child_mortality': 'education-impact/datasets/child_mortality_0_5_year_olds_dying_per_1000_born.csv',
 'children_per_woman': 'education-impact/datasets/children_per_woman_total_fertility.csv',
 'co2_emissions': 'education-impact/datasets/co2_emissions_tonnes_per_person.csv',
 'expenditure_per_student': 'education-impact/datasets/expenditure_per_student_secondary_percent_of_gdp_per_person.csv',
 'food_supply': 'education-impact/datasets/food_supply_kilocalories_per_person_and_day.csv',
 'gdp_per_captia': 'education-impact/datasets/gdp_per_capita_yearly_growth.csv',
 'gini_index': 'education-impact/datasets/gini.csv',
 'infant_mortality': 'education-impact/datasets/Infant_Mortality_Rate.csv',
 'life_expectancy': 'education-impact/datasets/life_expectancy_years.csv',
 'malnutrition': 'education-impact/datasets/malnutrition_weight_for_age_percent_of_children_under_5.csv',
 'maternal_mortality': 'education-impact/datasets/mmr_who.csv',
 'people_in_poverty': 'education-impact/datasets/numbe

In [ ]:
from pandas import read_csv
df = read_csv(datasets_path["child_mortality"])
print(df.head)
data = df.values
data

<bound method NDFrame.head of                   country   1960   1961   1962  ...    2012   2013   2014   2015
0             Afghanistan  357.0  351.0  345.0  ...   80.50  77.00  73.50  70.40
1                  Angola  308.0  305.0  302.0  ...  105.00  98.40  92.90  88.10
2                 Albania  188.0  180.0  173.0  ...   11.40  10.70  10.10   9.60
3                 Andorra   36.6   34.5   32.6  ...    3.79   3.61   3.43   3.27
4    United Arab Emirates  200.0  189.0  180.0  ...    8.13   8.00   7.91   7.83
..                    ...    ...    ...    ...  ...     ...    ...    ...    ...
192                 Samoa  135.0  131.0  127.0  ...   18.30  17.90  17.60  17.20
193                 Yemen  418.0  414.0  409.0  ...   55.00  55.00  55.00  55.00
194          South Africa  191.0  188.0  184.0  ...   43.30  41.20  39.50  37.70
195                Zambia  206.0  202.0  198.0  ...   74.30  70.70  66.90  64.60
196              Zimbabwe  151.0  147.0  142.0  ...   69.80  62.30  57.50  54.3

array([['Afghanistan', 357.0, 351.0, ..., 77.0, 73.5, 70.4],
       ['Angola', 308.0, 305.0, ..., 98.4, 92.9, 88.1],
       ['Albania', 188.0, 180.0, ..., 10.7, 10.1, 9.6],
       ...,
       ['South Africa', 191.0, 188.0, ..., 41.2, 39.5, 37.7],
       ['Zambia', 206.0, 202.0, ..., 70.7, 66.9, 64.6],
       ['Zimbabwe', 151.0, 147.0, ..., 62.3, 57.5, 54.3]], dtype=object)